In [1]:
from recommender_class_test import Recommender

In [2]:
r = Recommender('burgos', 'bmw', 'ciclomotor', 1987, 30)

In [3]:
r.recommender()

,city,brand,model,type,color,km,year,price,url,total_metric_pond
9587,burgos,ktm,sx_65,infantil,naranja,50.0,2016,3200,http://motos.net/ocasion/ktm/sx_65/2016-en-bur...,20.500000
763,vizcaya,ktm,sx_65,infantil,naranja,1.0,2012,2200,http://motos.net/ocasion/ktm/sx_65/2012-en-viz...,32.202584
20381,navarra,ktm,sx_65,infantil,not specified,200.0,2015,2999,http://motos.net/ocasion/ktm/sx_65/2015-en-nav...,36.032255
14000,madrid,kawasaki,kx_80,infantil,not specified,1000.0,1998,1500,http://motos.net/ocasion/kawasaki/kx_80/1998-e...,39.304038
11180,madrid,ktm,50_senior_adventure,infantil,naranja,NaN,2004,999,http://motos.net/ocasion/ktm/50_senior_adventu...,40.704038
14585,madrid,ktm,sx_50_lc_pro_senior,infantil,naranja,1000.0,2006,1190,http://motos.net/ocasion/ktm/sx_50_lc_pro_seni...,40.904038
20773,salamanca,ktm,sx_65,infantil,blanca-amarilla,100.0,2014,2750,http://motos.net/ocasion/ktm/sx_65/2014-en-sal...,42.725753
20808,salamanca,ktm,sx_50,infantil,not specified,NaN,2017,3410,http://motos.net/ocasion/ktm/sx_50/2017-en-sal...,43.025753
20923,alava,bmw,c_650_gt,scooters_+125cc,not specified,15000.0,2012,6500,http://motos.net/ocasion/bmw/c_650_gt/2012-en-...,48.512047
22778,cantabria,bmw,c_600_sport,scooters_+125cc,azul,15000.0,2014,7990,http://motos.net/ocasion/bmw/c_600_sport/2014-...,48.645768


In [ ]:
r.recommender().values.tolist()

In [ ]:
import pandas as pd
df_motos_raw = pd.read_csv('scraped_data/motos_raw_data.csv', sep=';')
df_motos_raw.shape

In [ ]:
df_motos_raw['color'].fillna('not specified', inplace=True)

In [ ]:
df_motos_raw.shape

Data cleaning

In [ ]:
# Convert the dataframe to lower case
df_motos_raw = df_motos_raw.apply(lambda x: x.astype(str).str.lower())

In [ ]:
#Join the words in the 'model' and 'type' fields with an underscore
df_motos_raw['model'] = df_motos_raw['model'].str.replace(' ', '_')
df_motos_raw['type'] = df_motos_raw['type'].str.replace(' ', '_')

In [ ]:
# Looking for duplicates
df_motos_raw['is_duplicated'] = df_motos_raw.duplicated()
duplicates = df_motos_raw['is_duplicated'].sum()
print '%d duplicates' %duplicates

In [ ]:
# Removing duplicates and delete 'is_duplicated' column
df_motos_raw = df_motos_raw.loc[df_motos_raw['is_duplicated']==False]
df_motos_raw = df_motos_raw.drop('is_duplicated', 1)
df_motos_raw.shape

In [ ]:
# Lets investigate column by column the NaNs we have in the dataframe
for column in df_motos_raw.columns:
    n_nan = df_motos_raw[column]=='nan'
    print column + " %d -- > %f" %(n_nan.sum(), (n_nan.sum()*1.0)/df_motos_raw.shape[0]*100)

In [ ]:
# Lets investigate the unique values we have in the columns
for column in ['city', 'brand', 'model', 'type', 'color', 'year']:
    column_uv = df_motos_raw[column].unique()
    print column + " --> " + "%d unique values" %len(sorted(column_uv))

Merge some data

In [ ]:
#In order to calculate our first metric, we will use the following columns:
# "lon" and "lat" : These are the longitude and latitude of the corresponding city. We will add them later.
# "brand", "model", "type", "year"

In [ ]:
# Calculating the longitude and latitude of the cities
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [ ]:
cities = df_motos_raw['city'].unique()
locations_rows = []
for city in cities:
    location = geolocator.geocode([city], timeout = 15)
    locations_rows.append([city, location.latitude, location.longitude])
#Save into a csv
df_locations = pd.DataFrame(locations_rows, columns = ['city', 'lat', 'lon'])
#df_locations.to_csv('auxiliary_data/locations_coords.csv', index = False)

In [ ]:
df_locations.head(3)

In [ ]:
# Merge df_locations with df_motos_raw
df_motos_raw_coord = pd.merge(df_motos_raw, df_locations, on = 'city')
#Save into a csv
#df_motos_raw_coord.to_csv('df_motos_raw_coord.csv', index = False)

In [ ]:
# We have created two csv files with a rank for the motos brands and types
# called rank_motos_brands.csv and rank_motos_types.csv

# With the first one, rank_motos_brands.csv, we are gint to create another  column, with a score to the corresponding brand
df_rank_moto_brand = pd.read_csv('scraped_data/rank_moto_brands.csv', sep=';')
df_motos_raw_coord_brand = pd.merge(df_motos_raw_coord, df_rank_moto_brand, on = 'brand', how = 'left')
#If the brand does not exist, the rank value will be zero
#!!OJO, AÑADIR MARCAS DE MOTOS QUE FALTAN ANTES DE PONER UN CERO!!!!!!!!!!!!!!!!!
df_motos_raw_coord_brand.brand_score.fillna(0, inplace=True)
# Save into a csv
df_motos_raw_coord_brand.to_csv('df_motos_coord_brand.csv', index = False)

# With the second one, rank_moto_types.csv, we are going to create another column, with a score to the corresponding type
df_rank_moto_type = pd.read_csv('auxiliary_data/rank_moto_types.csv', sep=';')
df_motos_raw_coord_brand_types = pd.merge(df_motos_raw_coord_brand, df_rank_moto_type, on = 'type', how = 'left')
#Save into a csv
df_motos_raw_coord_brand_types.to_csv('df_motos_raw_coord_brand_type.csv', index = False)

In [ ]:
df_motos_raw_coord_brand_types.head(3)

In [ ]:
sorted(df_motos_raw_coord_brand_types.brand.unique())

In [ ]:
# OK!, so we have a first version of the data we will use to recommend vehicles
# df_motos_raw_coord_brand_types
# Lets try to calculate the metric only with some fields. We are going to add these distances:
# cities distance, brand_distance, type_distance, year_distance
# We need to create some functions:

def cities_distance(city_lat, city_lon, user_lat, user_lon):
    """    
    :param city_lat: the value in the dataset's lat column to the corresponding city
    :param city_lon: the value in the dataset's lon column to the corresponding city
    :param user_lat: The corresponding lat value in the location dataset of the city selected by the user
    :param user_lon: The corresponding lon value in the location dataset of the city selected by the user
    
    :return: The value in kilometers of the distance between the two cities.
    
    Usage of the Vicenty distance
    """
    
    from geopy.distance import vincenty
    
    column_city = (city_lat, city_lon)
    user_city = (user_lat, user_lon)
    
    return (vincenty(column_city, user_city).km)

def distance_abs_value(a_value, b_value):
    return abs(a_value - b_value)

def w_s(city_row, brand_row, type_row, year_row):
    import numpy as np
    weigth = 100
    brand_weight = 40
    type_weight = 40
    year_weight = 10
    city_weight = 10
    
    params = np.array([city_row, brand_row, type_row, year_row])
    weights = np.array([city_weight, brand_weight, type_weight, year_weight])
    
    num = sum(params * weights) * 1.0
    return num/weigth

In [ ]:
#Example of request of a user
user_request = ('madrid', 'bmw', 'custom', 2000)

# We need to calculate some variables:
user_lat = float(df_locations[df_locations['city'] == user_request[0]].lat)
user_lon = float(df_locations[df_locations['city'] == user_request[0]].lon)
user_brand = int(df_rank_moto_brand[df_rank_moto_brand['brand'] == user_request[1]].brand_score)
user_type = int(df_rank_moto_type[df_rank_moto_type['type'] == user_request[2]].type_score)
user_year = user_request[3]

user_vars = [user_brand, user_type, user_year]
score_columns = ['brand_score', 'type_score', 'year']

In [ ]:
df_motos_raw_coord_brand_types['city_metric'] = df_motos_raw_coord_brand_types.apply(lambda row: cities_distance(row['lat'], row['lon'], user_lat, user_lon), axis=1)

for i, element in enumerate(['brand', 'type', 'year']):
    new_column = element + '_metric'
    print new_column, score_columns[i], user_vars[i]
    df_motos_raw_coord_brand_types[new_column] = df_motos_raw_coord_brand_types.apply(lambda row: distance_abs_value(int(row[score_columns[i]]), user_vars[i]), axis=1)
    
#!!! Revisar ranking de motos, falta alguna, por eso aparenden NAN en la columna brand_score al hacer el merge

df_motos_raw_coord_brand_types['total_metric_pond'] = df_motos_raw_coord_brand_types.apply(lambda row: w_s(row['city_metric'], row['brand_metric'], row['type_metric'], row['year_metric']), axis = 1)

In [ ]:
results = df_motos_raw_coord_brand_types.sort_values(by = ['total_metric_pond'], ascending=True)

In [ ]:
results.head(10)

In [ ]:
df_motos_raw_coord_brand_types.type.unique()

In [ ]:
df_motos_raw_coord_brand_types[df_motos_raw_coord_brand_types.type=='con_marchas']

In [ ]:
import pandas as pd
test = pd.read_csv('df_motos_raw_coord_brand_type.csv', sep=',', encoding = 'utf-8')

In [ ]:
if df_rank_moto_brand[df_rank_moto_brand['brand'] == 'periquito'].brand_score.empty:
    print "none"

In [ ]:
user_brand

In [ ]:
brand_in_df_list = [(brand, 'N/A') for brand in sorted(df_motos_raw_coord_brand_types.brand.unique())]

In [ ]:
df_brand_in_df_list = pd.DataFrame.from_records(brand_in_df_list, columns = ['brand', 'brand_score'])

In [ ]:
df_brand_in_df_list.head(5)

In [ ]:
df_rank_moto_brand.head()

In [ ]:
df_rank_final = pd.merge(df_brand_in_df_list, df_rank_moto_brand, on = 'brand', how = 'left')

In [ ]:
df_rank_final.head(5)

In [ ]:
rank_values = df_rank_moto_brand.values.tolist()
brands_rank_values =  df_rank_moto_brand.brand.values
for df_brand in brand_in_df_list:
    if df_brand[0] not in brands_rank_values:
        rank_values.append([df_brand[0],0])

In [ ]:
rank_values

In [4]:
from flask.ext.bootstrap import Bootstrap

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: ExtDeprecationWarning: Importing flask.ext.bootstrap is deprecated, use flask_bootstrap instead.
  """Entry point for launching an IPython kernel.


In [5]:
from flask_bootstrap import Bootstrap